In [17]:
import numpy as np
import pandas as pd   
import scipy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [18]:
cg = 1.78
eps = 1e-2
laplace_scale = 1.0 / (2**.5)

In [19]:
np.random.seed(50)

## Helper Functions

In [20]:
def random_generator(n,a,gamma=(2*cg)**.5):
    """Draws random numbers from normal, laplace and cauchy distributions."""
    distributions = {}
    distributions['normal'] = a + np.random.randn(n)
    distributions['laplace'] = a + np.random.laplace(size = n,scale = laplace_scale)
    distributions['cauchy'] = a + scipy.stats.cauchy.rvs(size=n,loc = 0, scale = gamma)
    return distributions

def f(data,gamma,a):
    """Optimization function for the cauchy distribution (given gamma)."""
    return np.sum((a-data)/(1 + ((data-a)/gamma)**2))

def df_da(data,gamma,a):
    """derivative of f with respect to a."""
    return np.sum((1 - ((data-a)/gamma)**2)/((1 + ((data-a)/gamma)**2)**2))

def nr(data,guess,gamma,iterations=1000):
    """Newton-Raphson method for estimating the scale parameter of a Cauchy distribution."""
    for iteration in range(iterations):
        y = f(data,gamma, guess)
        dy_dA = df_da(data, gamma, guess)
        guess -= y/dy_dA
        if abs(y) < eps:
            break
    return guess

def grad_desc(data,guess,gamma,iterations=1000):
    """gradient descent method for estimating the location parameter of a cauchy distribution."""
    # print(guess)
    lr = 1e-3
    pguess = 1e5
    for _ in range(iterations):
        y = f(data,gamma, guess)
        # dy_da = df_da(data, gamma, guess)
        pguess = guess
        guess -= lr*y
        if abs(pguess-guess) < eps:
            break
        # print(guess)
    # print("over")
    return guess

def mle_estimates(distributions, gamma, guess):
    """Calculates the mle estimates for normal, laplace and cauchy distributions."""
    estimates = {}
    for key in distributions.keys():
        if key == 'normal':
            estimates[key] = np.mean(distributions[key])
        elif key == 'laplace':
            estimates[key] = np.median(distributions[key])
        elif key == 'cauchy':
            estimates[key] = nr(distributions[key], guess, gamma=gamma,iterations=10000)
    return estimates



## Experiments

In [21]:
# from tqdm import tqdm

def realisations(n, a, num_realisations):
    """Generates random numbers and calculates MLE estimates for multiple realisations."""
    estimates = {"normal": [], "laplace": [], "cauchy": []}
    
    for _ in tqdm(
        range(num_realisations),
        desc=f"n={n} | Realisations",
        leave=False,
        dynamic_ncols=True,
    ):
        distributions = random_generator(n, a)
        estimate = mle_estimates(distributions, gamma=(2 * cg) ** 0.5, guess=np.median(distributions["cauchy"]))

        for key in estimate:
            estimates[key].append(estimate[key])
    
    return estimates

def get_statistics(estimates):
    """Calculates mean and standard deviation of MLE estimates."""
    statistics = {}
    for key in estimates:
        statistics[key] = {
            'mean': np.mean(estimates[key]),
            'var': np.var(estimates[key])
        }
    return statistics

def iter_over_sample_size(ns, a, num_realisations):
    """Iterates over different sample sizes and calculates MLE estimates."""
    estimates = {}
    stats = {}

    outer = tqdm(ns, desc="Sample Sizes", dynamic_ncols=True)
    for n in outer:
        outer.set_description(f"Sample size: n={n}")
        estimates[n] = realisations(n, a, num_realisations)
        stats[n] = get_statistics(estimates[n])
        outer.write(f"Done: n={n}, num_real={num_realisations}")
    
    return estimates, stats

def give_dataframe(a, sample_sizes, num_realisations):
    data = {}
    for ni in sample_sizes:
        data[f"{ni}"]={}
        for nri in num_realisations:
            data[f"{ni}"][f"normal_{nri}"]={}
            data[f"{ni}"][f"laplace_{nri}"]={}
            data[f"{ni}"][f"cauchy_{nri}"]={}
            
    for nr in num_realisations:
        _, stats = iter_over_sample_size(sample_sizes, a, nr)
        for ni in sample_sizes:
            data[f"{ni}"][f"normal_{nr}"] = rf"{stats[ni]["normal"]["mean"]:.3f} ± {np.sqrt(stats[ni]["normal"]["var"]):.3f}"
            data[f"{ni}"][f"laplace_{nr}"] = rf"{stats[ni]["laplace"]["mean"]:.3f} ± {np.sqrt(stats[ni]["laplace"]["var"]):.3f}"
            data[f"{ni}"][f"cauchy_{nr}"] = rf"{stats[ni]["normal"]["mean"]:.3f} ± {np.sqrt(stats[ni]["cauchy"]["var"]):.3f}"
    df = pd.DataFrame(data).T
    ordered_cols = [f"normal_{nr}" for nr in num_realisations]
    ordered_cols.extend([f"laplace_{nr}" for nr in num_realisations])
    ordered_cols.extend([f"cauchy_{nr}" for nr in num_realisations])
    df = df.loc[:,ordered_cols]
    return df


## Plotting functions

In [22]:
def plot_cdf(data, stat ,label='CDF', color='blue', axis=None):
    """Plots the CDF of the data."""
    if axis is None:
        _, axis = plt.subplots(figsize=(5, 5), layout="constrained")
    sorted_data = np.sort(data)
    cdf = np.linspace(0, 1, len(sorted_data))
    axis.step(sorted_data, cdf, where='post', label=label, color=color)
    axis.set_xlabel(r'Value of $\hat{a}$')
    axis.set_ylabel('Cumulative Probability')
    axis.set_title(f"CDF of {label} with mean: {stat['mean']:.3E} var: {stat['var']:.3E}")
    axis.set_xlim(max(-2,np.min(data)), min(4,np.max(data)))
    # plt.legend()
    # plt.show()

def plot_cdf_pdf(ns,estimates,stats):
    for n in ns:
        _, axs = plt.subplots(3, 2, figsize=(10, 5), layout="constrained")
        for i,key in enumerate(estimates[n].keys()):
            axs[i,0].hist(estimates[n][key], density=True, bins=100, alpha=0.5, color='blue', label='PDF',range=(max(-2,np.min(estimates[n][key])), min(4,np.max(estimates[n][key]))))
            axs[i,0].set_title(f"PDF of {key} with sample size {n}")
            plot_cdf(estimates[n][key], stats[n][key], label=key, color='blue', axis=axs[i,1])
            axs[i,1].set_title(f"CDF of {key} with sample size {n}")
        plt.show()

def plot_mean_var(stats):
    _, axs = plt.subplots(1, 2, figsize=(10, 5), layout="constrained")
    mean, var = {"normal": [], "laplace": [], "cauchy": []},{"normal": [], "laplace": [], "cauchy": []}
    for n in stats.keys():
        for key in mean.keys():
            mean[key].append(stats[n][key]['mean'])
            var[key].append(stats[n][key]['var'])
    for key in mean.keys():
        axs[0].semilogy(stats.keys(), mean[key], label=key)
        axs[1].semilogy(stats.keys(), var[key], label=key)
    axs[0].set_xlabel(r'Sample Size')
    axs[0].set_ylabel(r'Mean of $\hat{a}$')
    axs[0].set_title(r'Mean of $\hat{a}$')
    # axs[0].set_xscale(r'log')
    axs[1].set_xlabel(r'Sample Size')
    axs[1].set_ylabel(r'Variance of $\hat{a}$')
    axs[1].set_title(r'Variance of $\hat{a}$')
    # axs[1].set_xscale(r'log')
    axs[0].legend()
    axs[1].legend()
    plt.show()

In [23]:
n = [1, 10, 100, 1000, 10000]#, 1000, 10000]
num_realisations = [10,100,1000]

In [24]:
a = 1.0
df1 = give_dataframe(a, n, num_realisations)
df1.head()

Sample Sizes:   0%|          | 0/5 [00:00<?, ?it/s]

n=1 | Realisations:   0%|          | 0/10 [00:00<?, ?it/s]

Done: n=1, num_real=10


n=10 | Realisations:   0%|          | 0/10 [00:00<?, ?it/s]

Done: n=10, num_real=10


...

n=1000 | Realisations:   0%|          | 0/1000 [00:00<?, ?it/s]

Done: n=1000, num_real=1000


n=10000 | Realisations:   0%|          | 0/1000 [00:00<?, ?it/s]

Done: n=10000, num_real=1000


,normal_10,normal_100,normal_1000,laplace_10,laplace_100,laplace_1000,cauchy_10,cauchy_100,cauchy_1000
1,1.406 ± 1.262,1.076 ± 1.025,0.989 ± 1.028,1.198 ± 0.713,1.022 ± 0.817,1.044 ± 1.011,1.406 ± 52.802,1.076 ± 8.651,0.989 ± 1354.859
10,0.920 ± 0.330,1.028 ± 0.319,1.026 ± 0.308,1.007 ± 0.128,1.024 ± 0.301,0.989 ± 0.276,0.920 ± 1.071,1.028 ± 737.034,1.026 ± 1181.991
100,1.047 ± 0.091,0.977 ± 0.108,1.002 ± 0.101,0.994 ± 0.093,1.008 ± 0.068,1.001 ± 0.074,1.047 ± 0.196,0.977 ± 0.247,1.002 ± 0.266
1000,1.008 ± 0.023,1.000 ± 0.029,0.999 ± 0.032,0.992 ± 0.019,1.001 ± 0.023,1.001 ± 0.023,1.008 ± 0.057,1.000 ± 0.085,0.999 ± 0.085
10000,1.003 ± 0.012,1.000 ± 0.011,1.000 ± 0.010,0.999 ± 0.007,1.000 ± 0.007,1.000 ± 0.007,1.003 ± 0.026,1.000 ± 0.023,1.000 ± 0.025


In [25]:
a = 10.0
df10 = give_dataframe(a, n, num_realisations)
df10.head()

Sample Sizes:   0%|          | 0/5 [00:00<?, ?it/s]

n=1 | Realisations:   0%|          | 0/10 [00:00<?, ?it/s]

Done: n=1, num_real=10


n=10 | Realisations:   0%|          | 0/10 [00:00<?, ?it/s]

Done: n=10, num_real=10


...

n=1000 | Realisations:   0%|          | 0/1000 [00:00<?, ?it/s]

Done: n=1000, num_real=1000


n=10000 | Realisations:   0%|          | 0/1000 [00:00<?, ?it/s]

Done: n=10000, num_real=1000


,normal_10,normal_100,normal_1000,laplace_10,laplace_100,laplace_1000,cauchy_10,cauchy_100,cauchy_1000
1,10.140 ± 1.013,9.886 ± 0.937,9.991 ± 0.988,10.134 ± 0.771,9.887 ± 0.990,10.006 ± 1.000,10.140 ± 2.145,9.886 ± 28.592,9.991 ± 88.591
10,10.136 ± 0.302,9.980 ± 0.309,10.013 ± 0.313,10.020 ± 0.304,10.039 ± 0.227,10.019 ± 0.266,10.136 ± 3178.535,9.980 ± 807.438,10.013 ± 1134.997
100,10.016 ± 0.112,10.007 ± 0.102,9.999 ± 0.100,9.991 ± 0.106,10.010 ± 0.078,9.998 ± 0.075,10.016 ± 0.298,10.007 ± 0.270,9.999 ± 0.278
1000,9.992 ± 0.031,9.996 ± 0.033,10.000 ± 0.031,10.019 ± 0.021,9.999 ± 0.024,9.999 ± 0.022,9.992 ± 0.079,9.996 ± 0.079,10.000 ± 0.086
10000,10.001 ± 0.011,10.000 ± 0.009,10.000 ± 0.010,10.001 ± 0.007,10.000 ± 0.006,10.000 ± 0.007,10.001 ± 0.036,10.000 ± 0.026,10.000 ± 0.027


In [ ]:
plot_mean_var(stats_10)

In [ ]:
plot_cdf_pdf(n, estimates_1, stats_1)